In [ ]:
!pip install datasets
!pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 29.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-non

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
import numpy as np
from google.colab import drive
import torch


In [ ]:
!pip install accelerate>=0.21.0


In [ ]:
# Cargar los datos de entrenamiento
t_data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', usecols=[0, 1, 2, 3], nrows=1800, engine='python')

# Cargar los datos de validación
v_data = pd.read_csv('SMM4H_2024_Task3_Validation_600_codalab.csv', usecols=[0, 1, 2, 3], nrows=600, engine='python')

t_data = t_data.dropna(subset=['text', 'label'])
v_data = v_data.dropna(subset=['text', 'label'])


In [ ]:
print(t_data)
print(v_data)

           id                                keyword  \
0      3u2w5k                                    run   
1      3xbury                                outside   
2      3y743u                       run, swim, climb   
3      43bvs7                                   walk   
4      442ap2                                outside   
...       ...                                    ...   
1795   gqzye9                     pool, beach,  pool   
1796   env299                     outside , outdoors   
1797  e9bnr1s                                Jogging   
1798   qrmhbe                  walk, swimming,  pool   
1799   mxbsm8  roller blade, outside , roller blades   

                                                   text  label  
0     21/m. I want to experience young love, but I'v...      0  
1     Having issues talking to a girl whom I enjoyed...      0  
2     Need some advice for free social activities. I...      0  
3      I spoke to her today.. A few weeks ago I met ...      0  
4 

In [ ]:
# Filtrar los datos para eliminar las etiquetas con valor 0
train_data = t_data[t_data['label'] != 0]
val_data = v_data[v_data['label'] != 0]

print(train_data)
print(val_data)


           id                                keyword  \
1131  d3moq94                                   walk   
1132  d51rpnb                          outside, bike   
1133  d5bzg04                                   walk   
1134  d5ttkj7                                    run   
1135  d6se5to              running, run, horse, walk   
...       ...                                    ...   
1795   gqzye9                     pool, beach,  pool   
1796   env299                     outside , outdoors   
1797  e9bnr1s                                Jogging   
1798   qrmhbe                  walk, swimming,  pool   
1799   mxbsm8  roller blade, outside , roller blades   

                                                   text  label  
1131   Do you feel like the texts that you send back...      1  
1132   I'm gonna do the Pokemon thing to get myself ...      1  
1133   Something that work for me is to expose mysel...      1  
1134  Absolutely! Please encourage your son to do so...      1  
11

In [ ]:
# Definir una función para crear los conjuntos de datos
def create_datasets(train_texts, val_texts, train_labels, val_labels, tokenizer):
    train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
    val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})
    train_encodings = tokenizer(train_dataset['text'], truncation=True, padding=True)
    val_encodings = tokenizer(val_dataset['text'], truncation=True, padding=True)
    train_dataset = Dataset.from_dict({**train_encodings, 'label': train_dataset['label']})
    val_dataset = Dataset.from_dict({**val_encodings, 'label': val_dataset['label']})
    return train_dataset, val_dataset


In [ ]:
# Cargar el tokenizador y el modelo preentrenado
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", num_labels=3)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

In [ ]:
# Crear los conjuntos de datos
train_texts, val_texts = train_data['text'].tolist(), val_data['text'].tolist()
train_labels, val_labels = train_data['label'].tolist(), val_data['label'].tolist()
train_dataset, val_dataset = create_datasets(train_texts, val_texts, train_labels, val_labels, tokenizer)


In [ ]:
# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    learning_rate=1e-5,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy='epoch'
)


In [ ]:
# Función para calcular las métricas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return classification_report(labels, predictions, output_dict=True)


In [ ]:
# Entrenar el modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()


In [ ]:
# Evaluar el modelo en el conjunto de validación
eval_results = trainer.evaluate(eval_dataset=val_dataset)
print("Evaluation results:", eval_results)
